In [19]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [20]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44905
2,Huelva,Confirmados PDIA 14 días,496
3,Huelva,Tasa PDIA 14 días,"96,65413021026171"
4,Huelva,Confirmados PDIA 7 días,198
5,Huelva,Tasa PDIA 7 dias,"38,583705204902856"
6,Huelva,Total Confirmados,45104
7,Huelva,Curados,41746
8,Huelva,Fallecidos,410


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44905.0


/tmp/ipykernel_3356/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12425.0


/tmp/ipykernel_3356/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3356/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3356/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3356/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4495 personas en los últimos 7 días 

Un positivo PCR cada 1563 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44905.0,198.0,496.0,513170.0,38.583705,96.654130,26.0
Huelva-Costa,26576.0,144.0,344.0,289548.0,49.732687,118.805863,15.0
Condado-Campiña,13933.0,28.0,95.0,156231.0,17.922179,60.807394,7.0
Ayamonte,2031.0,28.0,46.0,21104.0,132.676270,217.968158,4.0
Cartaya,1931.0,37.0,78.0,20083.0,184.235423,388.388189,4.0
Huelva (capital),12425.0,32.0,92.0,143837.0,22.247405,63.961290,4.0
Sierra de Huelva-Andévalo Central,3975.0,23.0,49.0,67391.0,34.129186,72.710006,4.0
Rociana del Condado,664.0,5.0,7.0,7939.0,62.980224,88.172314,3.0
Almonte,2256.0,10.0,27.0,24507.0,40.804668,110.172604,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Nava (La),9.0,2.0,2.0,258.0,775.193798,775.193798,0.0
Higuera de la Sierra,65.0,3.0,7.0,1291.0,232.378002,542.215337,1.0
Cartaya,1931.0,37.0,78.0,20083.0,184.235423,388.388189,4.0
Aroche,127.0,6.0,9.0,3054.0,196.463654,294.695481,0.0
Puebla de Guzmán,174.0,1.0,9.0,3092.0,32.341527,291.073739,0.0
Sanlúcar de Guadiana,35.0,1.0,1.0,403.0,248.138958,248.138958,NaN
Isla Cristina,3094.0,24.0,53.0,21393.0,112.186229,247.744589,1.0
Cumbres Mayores,134.0,4.0,4.0,1749.0,228.702115,228.702115,1.0
Ayamonte,2031.0,28.0,46.0,21104.0,132.676270,217.968158,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puebla de Guzmán,174.0,1.0,9.0,3092.0,32.341527,291.073739,0.0,0.111111
Palos de la Frontera,1003.0,2.0,12.0,11742.0,17.032873,102.197241,0.0,0.166667
Cerro de Andévalo (El),196.0,1.0,5.0,2327.0,42.973786,214.868930,0.0,0.200000
Cortegana,291.0,1.0,4.0,4602.0,21.729683,86.918731,0.0,0.250000
Alosno,230.0,1.0,4.0,3933.0,25.425884,101.703534,0.0,0.250000
San Juan del Puerto,757.0,2.0,7.0,9411.0,21.251727,74.381043,0.0,0.285714
Condado-Campiña,13933.0,28.0,95.0,156231.0,17.922179,60.807394,7.0,0.294737
Punta Umbría,1255.0,3.0,10.0,15355.0,19.537610,65.125366,0.0,0.300000
Moguer,1930.0,5.0,15.0,21867.0,22.865505,68.596515,1.0,0.333333
